In [9]:
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub)

In [58]:
import csv
import datetime

# initializing the titles and rows list 
fields = [] 
rows = [] 

raw_timestamps_to_hw = dict()
raw_references = dict()
raw_questions_to_hw = dict()

filename = 'contributions.csv'

# reading csv file 
with open(filename, 'r', encoding="utf8") as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
      
    # extracting field names through first row 
    fields = next(csvreader) 
  
    # extracting each data row one by one 
    for row in csvreader: 
        rows.append(row) 

    curr_question = ""
    curr_text = ""
    curr_topic = ""
    curr_folder = ""
    curr_answer = ""
    timestamp = ""
    curr_number = 0
    curr_follow_up = 0
    endorsed = 0
    # This will help us take the most recent version because it always comes first
    already_filled = 0
    
    for row in rows[:]: 
        # parsing each column of a row 
        for col,field in zip(row, fields): 
            if(field == "Post Number" and curr_number != int(col)):
                curr_folders = curr_folder.split()
                for folder in curr_folders:
                    if folder[-1] == ';':
                            folder = folder[:-1]
                    if timestamp[-3:] == 'UTC':
                        timestamp = timestamp[:-4]
                    date_time_obj = datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
                    if(curr_question != "" and curr_answer != ""):
                        
                        for loc in find_all(curr_answer, "@"):
                            #print(curr_answer)
                            next_space = loc + 1
                            while(next_space <= len(curr_answer)-1 and curr_answer[next_space].isdigit()):
                                next_space = next_space + 1

                            question = -1
                            try:
                                question = int(curr_answer[(loc + 1):next_space])
                            except ValueError:
                                question = -1
                            #print(question)
                            if curr_number in raw_references:
                                raw_references_q = raw_references[curr_number]
                                raw_references_q.append(question)
                                raw_references.update({curr_number : raw_references_q})
                            else:
                                raw_references_q = []
                                raw_references_q.append(question)
                                raw_references.update({curr_number : raw_references_q})
                                
                            if question in raw_references:
                                raw_references_q = raw_references[question]
                                raw_references_q.append(curr_number)
                                raw_references.update({question : raw_references_q})
                            else:
                                raw_references_q = []
                                raw_references_q.append(curr_number)
                                raw_references.update({question : raw_references_q})
                        
                        if folder in raw_timestamps_to_hw:
                            raw_timestamps = raw_timestamps_to_hw[folder]
                            raw_timestamps.append(date_time_obj)
                            raw_timestamps_to_hw.update({folder : raw_timestamps})
                        else:
                            raw_timestamps = []
                            raw_timestamps.append(date_time_obj)
                            raw_timestamps_to_hw.update({folder : raw_timestamps})
                            
                        if folder in raw_questions_to_hw:
                            raw_questions = raw_questions_to_hw[folder]
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({folder : raw_questions})
                        else:
                            raw_questions = []
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({curr_folder : raw_questions})
                curr_number = int(col)
                curr_follow_up = 0
                already_filled = 0
                curr_answer = ""
                curr_topic = ""
                timestamp = ""
                curr_folder = ""
                endorsed = 0
                
            elif(field == "Submission HTML Removed"): #the question could also be in subject
                curr_text = col
                
            elif(field == "Folders"):
                curr_folder = col
                
            elif(field == "Created At"):
                timestamp = col
                
            elif(field == "Part of Post"):
                if(col == "started_off_question"):
                    if(already_filled == 0):
                        curr_question = curr_text
                    already_filled = 0
                elif(col == "updated_question"):
                    # update question entry to post and follow up numbers
                    if(already_filled == 0):
                        curr_question = curr_text
                        already_filled = 1
                        
                elif(col == "updated_i_answer"):
                    # update last entry to the post and follow up number
                    # what if it was appended? how do we update just the end?
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                        already_filled = 1
                    endorsed = 1
                elif(col == "started_off_i_answer"):
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                    already_filled = 0
                    endorsed = 1
                    
                elif(col == "updated_s_answer"):
                    # update last entry to the post and follow up number
                    # what if it was appended? how do we update just the end?
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                        already_filled = 1
                elif(col == "started_off_s_answer"):
                    # append to answer - if it was previously endorsed and were appending should we keep it endorsed?
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                    already_filled = 0
                    
                elif(col == "followup"):
                    # new entry for the new followup question 
                    
                    curr_folders = curr_folder.split()

                    for loc in find_all(curr_answer, "@"):
                        #print(curr_answer)
                        next_space = loc + 1
                        while(next_space <= len(curr_answer)-1 and curr_answer[next_space].isdigit()):
                            next_space = next_space + 1

                        question = -1
                        try:
                            question = int(curr_answer[(loc + 1):next_space])
                        except ValueError:
                            question = -1
                        #print(question)
                        if curr_number in raw_references:
                            raw_references_q = raw_references[curr_number]
                            raw_references_q.append(question)
                            raw_references.update({curr_number : raw_references_q})
                        else:
                            raw_references_q = []
                            raw_references_q.append(question)
                            raw_references.update({curr_number : raw_references_q})
                            
                        if question in raw_references:
                            raw_references_q = raw_references[question]
                            raw_references_q.append(curr_number)
                            raw_references.update({question : raw_references_q})
                        else:
                            raw_references_q = []
                            raw_references_q.append(curr_number)
                            raw_references.update({question : raw_references_q})
                    
                    for folder in curr_folders:
                        if folder[-1] == ';':
                            folder = folder[:-1]
                        if timestamp[-3:] == 'UTC':
                            timestamp = timestamp[:-4]
                        date_time_obj = datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
                        
                        if folder in raw_timestamps_to_hw:
                            raw_timestamps = raw_timestamps_to_hw[folder]
                            raw_timestamps.append(date_time_obj)
                            raw_timestamps_to_hw.update({folder : raw_timestamps})
                        else:
                            raw_timestamps = []
                            raw_timestamps.append(date_time_obj)
                            raw_timestamps_to_hw.update({folder : raw_timestamps})
                            
                        if folder in raw_questions_to_hw:
                            raw_questions = raw_questions_to_hw[folder]
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({folder : raw_questions})
                        else:
                            raw_questions = []
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({curr_folder : raw_questions})
                            
                    curr_question = ""
                    curr_answer = ""
                    curr_folder = ""
                    timestamp = ""
                    endorsed = 0
                    already_filled = 0
                        
                    curr_follow_up = curr_follow_up + 1
                    curr_question = curr_text
                    
                elif(col == "reply_to_followup"):
                    curr_answer = curr_answer + "\n" + curr_text

#print(raw_questions_to_hw)

{'other': ["Hi! I was registered half an hour ago for both the lecture and recitation and just tried to switch my recitation to better fit my schedule. I dropped my old recitation and when I tried to add a new one the, the system wouldn't let me. I would like to switch into 214, 216, 217, all of which Penn in Touch says are open. I would, at this point, also be happy to take my old spot back, which was in 211, which I believe is now closed. I am very concerned that without the recitation slot, Penn in Touch will soon delete my lecture spot, as well. Could someone please help me re-register for recitation?\xa0\n\nThank you!\nShira Silver\nPenn ID: 76417234", 'Thank you. I just want to clarify - I am not currently enrolled in any recitation since I tried to make the swap through Penn in Touch. I understand that this was my mistake. Will it still be possible to swap if I know which one I was in?', "Hello! Are we counted present if we respond to PollEverywhere during a different lecture th

In [12]:
import csv

hw_timestamps_filename = 'hw_timestamps.csv'
with open(hw_timestamps_filename, 'w', encoding="utf8") as hw_timestamps_file: 
    # creating a csv reader object 
    results_writer = csv.writer(hw_timestamps_file) 

    for key, value in raw_timestamps_to_hw.items(): 
        
        print(key)
        dates = ""
        for l in value:
            dates = dates + l.strftime('%m/%d/%Y') + ","
            print(l.strftime('%m/%d/%Y'))
        results_writer.writerow([key, dates])

other
08/30/2018
08/30/2018
08/31/2018
09/01/2018
09/03/2018
09/05/2018
09/05/2018
09/05/2018
09/07/2018
09/08/2018
09/08/2018
09/09/2018
09/09/2018
09/13/2018
09/14/2018
09/17/2018
09/18/2018
09/19/2018
09/22/2018
09/22/2018
09/22/2018
09/23/2018
09/23/2018
09/26/2018
09/26/2018
09/27/2018
10/04/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/05/2018
10/06/2018
10/06/2018
10/07/2018
logistics
08/30/2018
09/01/2018
09/01/2018
09/04/2018
09/05/2018
09/05/2018
09/05/2018
09/11/2018
09/12/2018
09/12/2018
09/24/2018
09/25/2018
09/15/2018
09/16/2018
09/17/2018
09/18/2018
11/21/2018
11/27/2018
12/11/2018
12/11/2018
09/18/2018
09/18/2018
09/19/2018
09/19/2018
09/19/2018
09/19/2018
09/21/2018
09/22/2018
09/22/2018
10/14/2018
10/14/2018
10/12/2018
10/12/2018
09/26/2018
10/03/2018
exam
08/31/2018
09/01/2018
09/18/2018
11/21/2018
11/27/2018
12/11/2018
12/11/2018
09/26/2018
10/02/2018
10/02/2018
10/02/2018
10/05/2018
10/05/2018
10/05/2018
10/12/2018
10/12/2018
10/05/2018
10/05/2018
10/11/2018


In [11]:
import operator

def Remove(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list 

for key, value in raw_references.items():
    without_duplicates = Remove(value)
    raw_references.update({key: without_duplicates})
   
raw_references_sorted = sorted(raw_references.items(), key=lambda v: len(v[1]))

for key in raw_references_sorted:
    print(key)
        

(3, [35])
(44, [46])
(68, [18])
(79, [5])
(40, [80])
(99, [-1])
(108, [-1])
(113, [69])
(69, [113])
(116, [100])
(100, [116])
(120, [49])
(123, [-1])
(135, [-1])
(140, [8])
(8, [140])
(143, [14])
(146, [16])
(150, [22])
(157, [38])
(106, [159])
(101, [159])
(138, [161])
(7, [161])
(163, [12])
(12, [163])
(172, [-1])
(181, [5])
(186, [9])
(189, [98])
(190, [92])
(197, [16])
(200, [14])
(203, [174])
(174, [203])
(213, [-1])
(21, [220])
(125, [220])
(223, [18])
(227, [225])
(226, [9])
(229, [22])
(242, [14])
(251, [-1])
(259, [254])
(254, [259])
(268, [260])
(260, [268])
(274, [27])
(283, [-1])
(296, [23])
(299, [298])
(298, [299])
(308, [141])
(141, [308])
(310, [-1])
(315, [309])
(309, [315])
(316, [16])
(324, [-1])
(327, [-1])
(341, [5])
(349, [27])
(355, [27])
(356, [32])
(265, [359])
(29, [359])
(370, [19])
(19, [370])
(373, [31])
(31, [373])
(383, [35])
(387, [13])
(13, [387])
(391, [278])
(278, [391])
(462, [394])
(396, [28])
(3685, [400])
(404, [28])
(405, [38])
(410, [23])
(411, 